# Imports

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import pickle

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Concatenate, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

from tensorflow.keras import backend as K
import gc

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

import torch
from RealESRGAN import RealESRGAN

from tqdm import tqdm

C:\Users\danie\AppData\Roaming\Python\Python312\site-packages\RealESRGAN\model.py:59: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast()


# Parameters

In [2]:
# Number of images to upscale by RealESRGAN
n_rows = 2500

In [3]:
# Resizing scale
scale = 2

In [4]:
# NN Parameters
epochs = 25
batch_size = 16

# Loading data

In [5]:
# Sample n_rows worth of data
work_df, _ = train_test_split(pd.read_csv("houses_preprocessed.csv"), train_size=n_rows, shuffle=True, random_state=42)

# Image Upscaling

In [6]:
# Load RealESRGAN_x4
model_id = "RealESRGAN_x4"
device = torch.device('cuda' if torch.cuda.is_available() else None)
model = RealESRGAN(device, scale=scale)
model.load_weights('weights/RealESRGAN_x4.pth', download=False) # Needs only be done once

# Create output directory
model_output_dir = os.path.join("Images Upscaled", f"{model_id}")
os.makedirs(model_output_dir, exist_ok=True)

C:\Users\danie\AppData\Roaming\Python\Python312\site-packages\RealESRGAN\model.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loadnet = torch.load(model_path)


In [7]:
# Upscaling Images
for index, row in tqdm(work_df.iterrows(), total=n_rows, desc="Upscaling Images"):   
    # Load the image to be upscaled
    input_image = Image.open(row["image"]).convert("RGB")

    # Predict using RealESRGAN
    RealESRGAN_resize = model.predict(input_image)
    
    # LANCZOS resizing
    # Calculate the upscale dimensions
    original_width, original_height = input_image.size
    output_width = original_width * scale
    output_height = original_height * scale
    traditional_resize = input_image.resize((output_width, output_height), Image.LANCZOS)
           
    # Save the result
    RealESRGAN_resize.save(os.path.join(model_output_dir, f"{index}_{row['price']}_RealESRGAN.jpg"))
    traditional_resize.save(os.path.join(model_output_dir, f"{index}_{row['price']}_lanczos.jpg"))

Upscaling Images: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2500/2500 [45:07<00:00,  1.08s/it]


# Experimental set up

## Train and Test the models on the same data partioning

In [8]:
# Split into train and test
train, test = train_test_split(work_df, test_size = 0.2, random_state=42)

# Train data
X_train_tab = train[['n_citi', 'bed', 'bath', 'sqft']].values 
X_train_img = train['image'] # pd
y_train = train['price']

# Test data ith compression cols
X_test_tab = test[['n_citi', 'bed', 'bath', 'sqft']].values 
X_test_img = test['image'] # pd
y_test = test['price']

# Print shapes
print("Training Data Shapes:")
print(f"Tabular features: {X_train_tab.shape}")
print(f"Image features: {X_train_img.shape}")
print(f"Target prices: {y_train.shape}")
print("\nTest Data Shapes:")
print(f"Tabular features: {X_test_tab.shape}")
print(f"Image features: {X_test_img.shape}")
print(f"Target prices: {y_test.shape}")

Training Data Shapes:
Tabular features: (2000, 4)
Image features: (2000,)
Target prices: (2000,)

Test Data Shapes:
Tabular features: (500, 4)
Image features: (500,)
Target prices: (500,)


# Neural Networks and Models

## Base NN and Resnet

In [9]:
def base_nn(image_shape=(311, 415, 3)):
    # Image processing branch
    img_input = Input(shape=image_shape, name='image_input')
    x = Conv2D(32, (3, 3), activation='relu')(img_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1)(x) # Regression output for price prediction

    # Define the model
    nn_model = Model(inputs=img_input, outputs=output)
    
    # Compile the model
    nn_model.compile(optimizer='adam',
                  loss='mae',
                  metrics=['mae', 'R2Score'])
    
    # Display model summary debug
    # nn_model.summary()

    return nn_model

In [10]:
def resnet_nn(image_shape=(311, 415, 3)):
    # Image processing branch with pre-trained ResNet50
    res_net = ResNet50(weights='imagenet', include_top=False, input_shape=image_shape)
   
    # Unfreeze only the last 10 layers of resnet (fine-tuning) 
    res_net.trainable = False 
    for layer in res_net.layers[-10:]:
        layer.trainable = True

    # Image processing branch
    img_input = Input(shape=image_shape, name='image_input')
    x = res_net(img_input)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1)(x) # Regression output for price prediction
    
    # Define the model
    res_net_model = Model(inputs=img_input, outputs=output)
    
    # Compile the model
    res_net_model.compile(optimizer='adam', 
                          loss='mae',
                          metrics=['mae', 'R2Score'])
    
    # Display model summary debug
    # res_net_model.summary()

    return res_net_model

In [11]:
'''
I did not write this code, the code is from: https://www.tensorflow.org/tutorials/load_data/images
It helps us train the NN more dynamically, it loads images on the go, such that not all RAM is used up.
It does try to maximise RAM usage this is basically what the tf.data.AUTOTUNE does.
'''

# Loads an image and normalizes it from [0,1]
def process_example(image_path, label):
    # Load raw bytes and convert to RGB
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Normalize image to [0, 1] and convert to float32
    image = tf.image.convert_image_dtype(image, tf.float32)

    return image, label


# Creates on the fly data sets to train/test the model, we need this to not exceed memory
def create_dataset(image_paths, labels, shuffle=True):
    # Convert to tensors
    image_paths = tf.convert_to_tensor(image_paths)
    labels = tf.convert_to_tensor(labels, dtype=tf.float32)

    # Build dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda img, lbl: process_example(img, lbl), num_parallel_calls=tf.data.AUTOTUNE)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset


def train_and_evaluate_nn(nn, 
                          X_train_img_paths, y_train,
                          X_test_img_paths, y_test,
                          verbose=1):

    # Dynamic dataset loading
    train_upscale = create_dataset(X_train_img_paths, y_train, shuffle=True) # Shuffle to break ordering
    test_upscale = create_dataset(X_test_img_paths, y_test, shuffle=False) # No shuffle, we arent learning, just predicting

    # Train and Test
    history = nn.fit(train_upscale, epochs=epochs, verbose=verbose)
    test_loss, test_mae, r2 = nn.evaluate(test_upscale, verbose=0)

    return history, test_loss, test_mae, r2

## Logistic regression

In [12]:
def train_and_evaluate_lin_model(model, X_train_tab, y_train, X_test_tab, y_test):
    # Train the model
    model.fit(X_train_tab, y_train)
    
    # Evaluate the model
    y_test_pred = model.predict(X_test_tab)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    
    return mae_test, r2

### Train and Evaluate

In [13]:
# Create NNs with tabular features = 4 (n_citi, bed, bath, sqft)
nn_base = base_nn()
nn_resnet = resnet_nn()
lin = LinearRegression()

In [14]:
def assert_image_shapes(expected_image_res, xtrain, xtest):
    # Check training set sample
    train_sample_path = xtrain.iloc[0] # Get first training image
    train_img = np.array(Image.open(train_sample_path))
    assert train_img.shape == expected_image_res
    
    # Check test set sample
    test_sample_path = xtest.iloc[0]  # Get first test image
    test_img = np.array(Image.open(test_sample_path))
    assert test_img.shape == expected_image_res

In [15]:
assert_image_shapes((311, 415, 3), X_train_img, X_test_img)

In [16]:
# NN
print("Training Base NN")
nn_base_hist, _, nn_base_mae, nn_base_r2 = train_and_evaluate_nn(nn_base, X_train_img, y_train, X_test_img, y_test)
print(f"NN Base MAE: {nn_base_mae:.0f}\nNN Base R2: {nn_base_r2:.2f}")

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Base NN
Epoch 1/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - R2Score: -3.2351 - loss: 646451.3750 - mae: 646451.3750
Epoch 2/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - R2Score: -0.1751 - loss: 279172.6562 - mae: 279172.6562
Epoch 3/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - R2Score: -0.1458 - loss: 268616.1250 - mae: 268616.1250
Epoch 4/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - R2Score: -0.1460 - loss: 275600.6562 - mae: 275600.6562
Epoch 5/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - R2Score: -0.1360 - loss: 274374.8125 - mae: 274374.8125
Epoch 6/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - R2Score: -0.1698 - loss: 278471.5000 - mae: 278471.5000
Epoch 7/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - R2Score: -0.1476 - loss: 288671.0625 - mae: 288671.0625
Epoch 8/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - R2Score: -0.1278 - loss: 274205.3125 - mae: 274205.3125
Epoch 9/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - R2Score: -0.1442 - loss: 275325.2812 - m

0

In [17]:
# Resnet
print("Training Resnet")
nn_resnet_hist, _, nn_resnet_mae, nn_resnet_r2 = train_and_evaluate_nn(nn_resnet, X_train_img, y_train, X_test_img, y_test)
print(f"Resnet MAE: {nn_resnet_mae:.0f}\nResnet R2: {nn_resnet_r2:.2f}")

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Resnet
Epoch 1/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 324s 3s/step - R2Score: -3.4810 - loss: 698828.4375 - mae: 698828.4375
Epoch 2/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 322s 3s/step - R2Score: -3.4772 - loss: 685783.6250 - mae: 685783.6250
Epoch 3/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 323s 3s/step - R2Score: -3.1858 - loss: 683771.9375 - mae: 683771.9375
Epoch 4/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 322s 3s/step - R2Score: -3.0321 - loss: 650369.5000 - mae: 650369.5000
Epoch 5/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 322s 3s/step - R2Score: -2.5197 - loss: 598489.8750 - mae: 598489.8750
Epoch 6/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 321s 3s/step - R2Score: -2.2229 - loss: 526674.0000 - mae: 526674.0000
Epoch 7/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 322s 3s/step - R2Score: -1.3663 - loss: 437473.5625 - mae: 437473.5625
Epoch 8/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 322s 3s/step - R2Score: -0.4750 - loss: 313160.4062 - mae: 313160.4062
Epoch 9/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 322s 3s/step - R2Score: -0.1236 - loss: 270224.9688 - ma

0

In [18]:
# LR
print("Training LR")
lr_mae, lr_r2 = train_and_evaluate_lin_model(lin, X_train_tab, y_train, X_test_tab, y_test)
print(f"LR MAE: {lr_mae:.0f}\nLR R2: {lr_r2:.2f}")

Training LR
LR MAE: 234918
LR R2: 0.40


# Comparison Upscaled Images

In [19]:
'''
method input should be x train, y train, x test and y test.
Then hardcopy and return the x train image with the upscale method
return train test split x and y upscaled method image
'''
def load_upscale_data(upscale_method, xtrain, xtest):   
    # Load image paths and prices
    image_path = f"Images Upscaled/RealESRGAN_x4"
    X_train_upscale = []
    y_train_upscale = []
    X_test_upscale = []
    y_test_upscale = []   

    # For each image in directory
    for img_file in os.listdir(image_path):
        # Only take either the images generated by either realesrgan or lanczos
        if upscale_method in img_file:
            # Images have format: "i_price_upscaleMethod.jpg")
            image_id, image_price, upscale_method_img = img_file.split('_')
            image_id, image_price = (int(image_id), int(image_price)) # Cast to int
            assert upscale_method == upscale_method_img.strip(".jpg")

            # If in X_train add to X_upscale, vice versa y
            if image_id in xtrain.index:
                X_train_upscale.append(image_path + "/" + img_file)
                y_train_upscale.append(image_price)
            elif image_id in xtest.index: 
                X_test_upscale.append(image_path + "/" + img_file)
                y_test_upscale.append(image_price)
            else:
                raise Exception("Image ID not found")
    
    # Ensure loaded images match the expected number of upscaled images
    assert len(X_train_upscale) + len(X_test_upscale) == n_rows
    assert len(y_train_upscale) + len(y_test_upscale) == n_rows

    return X_train_upscale, y_train_upscale, X_test_upscale, y_test_upscale

In [20]:
def create_comparison(llm, model_results):
    # DF Structure
    comparison_data = {
        'Model': [],
        'MAE': [],
        'R2': []
    }
    
    for model_name, mae, r2 in model_results:
        comparison_data['Model'].append(model_name)
        comparison_data['MAE'].append(round(mae))
        comparison_data['R2'].append(round(r2, 3))
    
    # Make into df
    comparison_df = pd.DataFrame(comparison_data).set_index("Model")
    
    # Display df
    print(f"Comparison of Models for {llm}")
    display(comparison_df)
    print()
    return comparison_df

In [21]:
def print_shapes(xtrain, ytrain, xtest, ytest):
    # Print shapes
    print("Training Data Shapes:")
    print(f"Features: {len(xtrain)}")
    print(f"Target prices: {len(ytrain)}")
    print("\nTest Data Shapes:")
    print(f"Features: {len(xtest)}")
    print(f"Target prices: {len(ytest)}")

In [22]:
def assert_image_shapes_lst(expected_image_res, xtrain, xtest):
    # Check training set sample
    train_sample_path = xtrain[0] # Get first training image
    train_img = np.array(Image.open(train_sample_path))
    assert train_img.shape == expected_image_res
    
    # Check test set sample
    test_sample_path = xtest[0]  # Get first test image
    test_img = np.array(Image.open(test_sample_path))
    assert test_img.shape == expected_image_res

In [23]:
import logging

# Supress retracing warning
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # '0' = all messages, '3' = fatal only
tf.get_logger().setLevel(logging.ERROR)

In [24]:
# Initialize a list to store results
model_results = []

# Store default results
model_results.append(('NN Base', nn_base_mae, nn_base_r2))
model_results.append(('NN Resnet', nn_resnet_mae, nn_resnet_r2))

In [25]:
# lanczos
# Load upscaled images
X_train_upscale_lanczos, y_train_upscale_lanczos, X_test_upscale_lanczos, y_test_upscale_lanczos = load_upscale_data('lanczos', X_train_img, X_test_img)

# Debug
print_shapes(X_train_upscale_lanczos, y_train_upscale_lanczos, X_test_upscale_lanczos, y_test_upscale_lanczos)
assert_image_shapes_lst((311*scale, 415*scale, 3), X_train_upscale_lanczos, X_test_upscale_lanczos)

# Create, Train and Evaluate the models
nn_base_upscale_lanczos = base_nn((311*scale, 415*scale, 3))
nn_resnet_upscale_lanczos = resnet_nn((311*scale, 415*scale, 3))

print("Training Base NN")
_, _, nn_base_upscale_mae_lanczos, nn_base_upscale_r2_lanczos = train_and_evaluate_nn(nn_base_upscale_lanczos, 
                                                                      X_train_upscale_lanczos, y_train_upscale_lanczos, 
                                                                      X_test_upscale_lanczos, y_test_upscale_lanczos)
print("Training Resnet NN")
_, _, nn_resnet_upscale_mae_lanczos, nn_resnet_upscale_r2_lanczos = train_and_evaluate_nn(nn_resnet_upscale_lanczos, 
                                                                                          X_train_upscale_lanczos, y_train_upscale_lanczos, 
                                                                                          X_test_upscale_lanczos, y_test_upscale_lanczos)

# Store results for both models
model_results.append(("NN Base Upscaled " + 'lanczos', nn_base_upscale_mae_lanczos, nn_base_upscale_r2_lanczos))
model_results.append(("NN Resnet Upscaled " + 'lanczos', nn_resnet_upscale_mae_lanczos, nn_resnet_upscale_r2_lanczos))

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Data Shapes:
Features: 2000
Target prices: 2000

Test Data Shapes:
Features: 500
Target prices: 500
Training Base NN
Epoch 1/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 764s 6s/step - R2Score: -3.1025 - loss: 649290.8125 - mae: 649290.8125
Epoch 2/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 891s 6s/step - R2Score: -0.1817 - loss: 285212.7500 - mae: 285212.7500
Epoch 3/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 832s 6s/step - R2Score: -0.1525 - loss: 283796.6562 - mae: 283796.6562
Epoch 4/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 834s 6s/step - R2Score: -0.1143 - loss: 272049.2188 - mae: 272049.2188
Epoch 5/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 853s 6s/step - R2Score: -0.1579 - loss: 281113.3125 - mae: 281113.3125
Epoch 6/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 818s 6s/step - R2Score: -0.1491 - loss: 270548.7812 - mae: 270548.7812
Epoch 7/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 873s 6s/step - R2Score: -0.1527 - loss: 285588.4375 - mae: 285588.4375
Epoch 8/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 845s 6s/step - R2Score: -0.1208 - loss: 276273.6250 - mae

0

In [26]:
# RealESRGAN
# Load upscaled images
X_train_upscale_RealESRGAN, y_train_upscale_RealESRGAN, X_test_upscale_RealESRGAN, y_test_upscale_RealESRGAN = load_upscale_data('RealESRGAN', 
                                                                                                                                   X_train_img, 
                                                                                                                                   X_test_img)

# Create, Train and Evaluate the models
nn_base_upscale_RealESRGAN = base_nn((311*scale, 415*scale, 3))
nn_resnet_upscale_RealESRGAN = resnet_nn((311*scale, 415*scale, 3))

# Debug
print_shapes(X_train_upscale_RealESRGAN, y_train_upscale_RealESRGAN, X_test_upscale_RealESRGAN, y_test_upscale_RealESRGAN)
assert_image_shapes_lst((311*scale, 415*scale, 3), X_train_upscale_RealESRGAN, X_test_upscale_RealESRGAN)

print("Training Base NN")
_, _, nn_base_upscale_mae_RealESRGAN, nn_base_upscale_r2_RealESRGAN = train_and_evaluate_nn(nn_base_upscale_RealESRGAN, 
                                                                      X_train_upscale_RealESRGAN, y_train_upscale_RealESRGAN, 
                                                                      X_test_upscale_RealESRGAN, y_test_upscale_RealESRGAN)
print("Training Resnet NN")
_, _, nn_resnet_upscale_mae_RealESRGAN, nn_resnet_upscale_r2_RealESRGAN = train_and_evaluate_nn(nn_resnet_upscale_RealESRGAN, 
                                                                                          X_train_upscale_RealESRGAN, y_train_upscale_RealESRGAN, 
                                                                                          X_test_upscale_RealESRGAN, y_test_upscale_RealESRGAN)

# Store results for both models
model_results.append(("NN Base Upscaled " + 'RealESRGAN', nn_base_upscale_mae_RealESRGAN, nn_base_upscale_r2_RealESRGAN))
model_results.append(("NN Resnet Upscaled " + 'RealESRGAN', nn_resnet_upscale_mae_RealESRGAN, nn_resnet_upscale_r2_RealESRGAN))

# Create a comparison of model results
create_comparison(model_id, model_results)

# Try to clear NN from memory
K.clear_session()
gc.collect()

Training Data Shapes:
Features: 2000
Target prices: 2000

Test Data Shapes:
Features: 500
Target prices: 500
Training Base NN
Epoch 1/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 858s 6s/step - R2Score: -3.3778 - loss: 653327.4375 - mae: 653327.4375
Epoch 2/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 926s 6s/step - R2Score: -0.1929 - loss: 280087.4375 - mae: 280087.4375
Epoch 3/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 861s 6s/step - R2Score: -0.1189 - loss: 276370.5000 - mae: 276370.5000
Epoch 4/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 790s 6s/step - R2Score: -0.1519 - loss: 296378.0000 - mae: 296378.0000
Epoch 5/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 837s 6s/step - R2Score: -0.1588 - loss: 269393.4375 - mae: 269393.4375
Epoch 6/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 844s 6s/step - R2Score: -0.1487 - loss: 269432.5000 - mae: 269432.5000
Epoch 7/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 954s 6s/step - R2Score: -0.1443 - loss: 276236.7188 - mae: 276236.7188
Epoch 8/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 813s 6s/step - R2Score: -0.1295 - loss: 266526.0312 - mae

,MAE,R2
Model,,
NN Base,305006,-0.134
NN Resnet,514309,-1.551
NN Base Upscaled lanczos,305251,-0.171
NN Resnet Upscaled lanczos,280873,0.097
NN Base Upscaled RealESRGAN,313847,-0.090
NN Resnet Upscaled RealESRGAN,448771,-0.618


0